In [ ]:
# Import packages
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# Print version
print(tf.__version__)

In [ ]:
# Create placeholders for data
def get_data(batch_size):
    
    X = tf.placeholder(tf.float32, shape=[batch_size, 784])
    Y = tf.placeholder(tf.float32, shape=[batch_size, 10])
    
    return(X,Y)

In [ ]:
# Define forward pass function
def forward(X, arch, batch_size):
    
    image = tf.reshape(X, [batch_size, 28, 28, 1])
    hidden_layer = image
    
    for size in arch:
        num_filters, dim_filters, stride = size[0], size[1], size[2]
        hidden_layer = layers.conv2d(hidden_layer, num_filters, dim_filters, stride, "Same")
        convOut= tf.nn.relu(hidden_layer)
    
    new_shape = tf.divide(tf.size(hidden_layer), batch_size)
    hidden_layer = tf.reshape(hidden_layer, [batch_size, new_shape])
    prbs = layers.fully_connected(hidden_layer, 10, tf.nn.softmax)
    
    return(prbs)

In [ ]:
# Define loss function
def get_loss(Y, probs):
    
    loss = tf.reduce_mean(-1*tf.reduce_sum(Y*tf.log(probs), reduction_indices=[1]))
    
    return(loss)

In [ ]:
# Get accuracy
def get_accuracy(prbs, ans):
    
    correct = tf.equal(tf.argmax(prbs,1), tf.argmax(ans,1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    return(accuracy)

In [ ]:
# Get graph
def get_graph(batch_size, num_epochs, arch, learning_rate):
    
    X, Y = get_data(batch_size)
    probs = forward(X, arch, batch_size)
    loss = get_loss(Y, probs)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    nodes = [X, Y, probs, loss, optimizer]
    
    return(nodes)

In [ ]:
# Define training method
def train_model(batch_size, num_epochs, nn_arch, learning_rate):
    
    # Get nodes
    graph_nodes = get_graph(batch_size, num_epochs, nn_arch, learning_rate)
    
    # Unpack
    X = graph_nodes[0] 
    Y = graph_nodes[1] 
    
    probs = graph_nodes[2] 
    loss = graph_nodes[3] 
    optimizer = graph_nodes[4] 
    
    # Get accuracy graph
    accuracy = get_accuracy(probs, Y)
    
    # Start session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    # Train
    total_acc = 0
    
    for i in range(num_epochs):
        print(i)
        imgs, labels = train.next_batch(batch_size)
        _, acc = sess.run([optimizer,accuracy], feed_dict={X: imgs, Y: labels})
        total_acc +=acc
        
    print('The training accuracy is {}'.format(total_acc/num_epochs))
    
    # Get predictions on dev. set
    total_acc = 0
    
    for i in range(num_epochs):
        imgs, labels = dev.next_batch(batch_size)
        acc = sess.run(accuracy, feed_dict={X: imgs, Y: labels})
        total_acc += acc
    
    print('The dev. accuracy is {}'.format(total_acc/num_epochs))
        
    
    # Get predictions on test set
    total_acc = 0
    
    for i in range(num_epochs):
        imgs, labels = test.next_batch(batch_size)
        total_acc+=sess.run(accuracy, feed_dict={X: imgs, Y: labels})
    
    print('The test accuracy is {}'.format(total_acc/num_epochs))

In [ ]:
# Execute
if __name__ == '__main__':
    
    # Set a seed to make sure results are reproducible
    tf.set_random_seed(1234)

    # Get data
    s3_endpoint = 'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/'
    data = input_data.read_data_sets('data/fashion', source_url = s3_endpoint, one_hot = True)
    
    # Get training, dev., and test sets
    train, dev, test = data.train, data.validation, data.test
    
    # Set parameters
    batch_size = 100
    num_epochs = 1000
    nn_arch = [[32, [2,2], 1], [16, [2,2], 1], [12, [2,2], 1], [10, [2,2], 1]]
    learning_rate = 0.2
    
    # Train
    train_model(batch_size, num_epochs, nn_arch, learning_rate)